<a href="https://colab.research.google.com/github/esequielsantos/ediBot/blob/main/EdiBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EdiBot
###Segue requisitos e uso da API
Instalar Gemini

---



In [ ]:
!pip install -U google-generativeai

## Set up my API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:


In [2]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos disponiveis


In [ ]:
for m in genai.list_tuned_models():
  print (m)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

Configurando o modelo
Temperaura:

In [18]:
gen_config = {
    "candidate_count": 1,
    "temperature": 1,
}

Segurança:

In [19]:
saf_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Usar **modelo** escolhido acima

In [20]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro-latest', safety_settings= saf_settings, generation_config= gen_config)

#Enviando mensagem ao modelo

In [ ]:
response = model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton")
#print(model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton").text)

##Exibindo Resposta

In [ ]:
print(response.text)

NameError: name 'response' is not defined

#EdiBOT

> Adicionar aspas INICIANDO O EdiBot

**Iniciando**



In [35]:
from datetime import datetime

*Rode o comando abaixo para limpar o historico do chat*

In [41]:
chat = model.start_chat(history=[])

In [113]:
def edibot(prompt, history):
  if not chat.history:
    agora = datetime.now()
    agorastr= formatted_string = f"{agora:%d/%m/%Y %H:%M:%S}"
    hora = agora.hour - 3
    periodo_dia = ""
    if 6 <= hora < 12:
      periodo_dia = "manhã"
    elif 12 <= hora < 18:
      periodo_dia = "tarde"
    else:
      periodo_dia = "noite"
    bemvindo = model.generate_content(f"Você é um atendente de uma Loja física de informatica chamada ByteMe, que vende computadores, laptops e acessórios chamado EdiBot. Escreva uma mensagem de saudação de {periodo_dia} ").text
    response = chat.send_message(prompt)
    return agorastr + '\n\n' + bemvindo + '\n\n-----------------------\n' + response.text
  else:
    prompt = prompt.lower()
    agradecer = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe. Escreva uma mensagem curta de agradecimento por ter entrado em contato").text
    if prompt == "history":
      markdown_text = ""
      for message in chat.history:
        sender = message.role
        text = message.parts[0]
        markdown_text += f"* **{sender}**: *{text}*\n"
      return markdown_text
    elif 'obrigado' in prompt:
      return agradecer
    elif 'tchau' in prompt:
      return agradecer
    elif 'fim' in prompt:
      return agradecer
    elif 'sair' in prompt:
      return agradecer
    else:
      response = chat.send_message(prompt)
      return response.text
  #continuar = model.generate_content("Escreva uma mensagem curta e agradavél perguntando se deseja mais alguma informação").text
  #return continuar

##Teste da funcao inteligente
*Exige parada total do GRadio*

In [ ]:
print(edibot("Bom dia, preciso de um computador novo para jogar", []))

In [ ]:
print(edibot("e se for pra usar Autocad ?", []))

In [114]:
print(edibot('history', []))
#chat.history

* **user**: *text: "Bom dia, preciso de um computador novo para jogar"
*
* **model**: *text: "**Recomenda\303\247\303\265es de Computadores para Jogos**\n\n**Faixa de Pre\303\247os M\303\251dia (R$ 3.000 - R$ 5.000)**\n\n* **Lenovo Legion 5 (15 polegadas)**\n  * Processador: AMD Ryzen 5 5600H\n  * Placa de v\303\255deo: NVIDIA GeForce GTX 1650\n  * RAM: 8 GB DDR4\n  * Armazenamento: SSD de 256 GB\n* **Dell G15 5515 (15 polegadas)**\n  * Processador: Intel Core i5-11400H\n  * Placa de v\303\255deo: NVIDIA GeForce GTX 1650 Ti\n  * RAM: 8 GB DDR4\n  * Armazenamento: SSD de 256 GB + HDD de 1 TB\n\n**Faixa de Pre\303\247os Alta (R$ 5.000 - R$ 8.000)**\n\n* **ASUS ROG Strix G15 (15 polegadas)**\n  * Processador: AMD Ryzen 7 5800H\n  * Placa de v\303\255deo: NVIDIA GeForce RTX 3050\n  * RAM: 16 GB DDR4\n  * Armazenamento: SSD de 512 GB\n* **MSI Katana GF66 (15 polegadas)**\n  * Processador: Intel Core i7-11800H\n  * Placa de v\303\255deo: NVIDIA GeForce RTX 3060\n  * RAM: 16 GB DDR4\n  * Arma

##Uso do GRADIO

In [ ]:
!pip install -U gradio

In [15]:
import gradio as gr

#Tela Gráfica para o BOT

In [ ]:
demo = gr.ChatInterface(edibot,
                 title='BatePapo BiteMe',
                 textbox=gr.Textbox(placeholder="Pergunte ao EdiBOT"),
                 retry_btn='Voltar',
                 clear_btn='Limpar',
                 undo_btn='Desfazer',
                 stop_btn='Parar',
                 submit_btn='Enviar [Enter]')

if __name__ == "__main__":
    demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f55dac2fe297d670f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Subir para o GitHub